In [1]:
import os
import pandas as pd
import numpy as np
import kagglehub
import shutil

In [2]:
# Download the dataset
path = kagglehub.dataset_download("abdulmalik1518/the-ultimate-cars-dataset-2024")

target_path = "../data/raw/"
os.makedirs(target_path, exist_ok=True)

# Find the CSV file in the downloaded directory
for file_name in os.listdir(path):
    if file_name.endswith(".csv"):
        old_file_path = os.path.join(path, file_name)
        file_path = os.path.join(target_path, "ultimate_cars_dataset_2024.csv")
        
        # Move and rename the file
        shutil.move(old_file_path, file_path)

In [3]:
# Load dataset
file_path = "../data/raw/ultimate_cars_dataset_2024.csv"  # Removed extra quote
cars_df = pd.read_csv(file_path, encoding="ISO-8859-1")

cars_df.head()


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Data Cleaning
- Missingness in the data- The missing data in this dataset is **very small (<0.5%)** and **randomly distributed (MCAR)**, with no correlation to other features. Since imputing values could introduce bias, **deleting these rows is the best approach**, ensuring clean and unbiased data for visualization.
- Range values (for example, horsepower within the range "70-85") are coverted to their average within the range for that specific observation.


In [4]:
notebook_dir = os.path.dirname(os.path.abspath("__file__"))  # Notebook's location
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # Go one level up

# Define paths for raw and processed data
raw_data_path = os.path.join(project_root, "data", "raw", "ultimate_car_dataset_2024.csv")
processed_dir = os.path.join(project_root, "data", "processed")

# Standardize column names
cars_df.columns = cars_df.columns.str.strip().str.lower().str.replace(" ", "_", regex=True).str.replace(r"[()\[\]]", "", regex=True)

# Extract numeric values for CC/Battery Capacity
cars_df['cc_battery_capacity'] = cars_df['cc/battery_capacity'].astype(str)
cars_df['battery_capacity_kwh'] = cars_df['cc_battery_capacity'].str.extract(r'([\d.]+)\s*kwh', expand=False).astype(float)
cars_df['cc_capacity'] = cars_df['cc_battery_capacity'].str.extract(r'([\d.]+)\s*cc', expand=False).astype(float)

# Create a binary indicator for electric vehicles
cars_df['is_electric'] = cars_df['battery_capacity_kwh'].notna().astype(int)

# Fill missing values appropriately
cars_df['cc_capacity'] = cars_df['cc_capacity'].fillna(0)
cars_df['battery_capacity_kwh'] = cars_df['battery_capacity_kwh'].fillna(0)

# Function to extract the minimum value from a range or clean numeric data
def extract_min_value(value):
    if pd.isna(value):
        return value
    value = str(value).lower().replace("cc", "").replace("~", "").replace("hp", "").replace("km/h", "").replace("sec", "").replace("nm", "").replace("$", "").replace(",", "").strip()
    if '-' in value:
        return float(value.split('-')[0])  # Take the minimum value in range
    try:
        return float(value)
    except ValueError:
        return None  # Return None if conversion fails

# Apply cleaning to numerical columns
cars_df['horsepower'] = cars_df['horsepower'].apply(extract_min_value)
cars_df['total_speed'] = cars_df['total_speed'].apply(extract_min_value)
cars_df['performance_0_100_km/h'] = cars_df['performance0_-_100_km/h'].apply(extract_min_value)
cars_df['torque'] = cars_df['torque'].apply(extract_min_value)
cars_df['cars_prices'] = cars_df['cars_prices'].apply(extract_min_value)

cars_df['seats'] = pd.to_numeric(cars_df['seats'], errors='coerce')

# Fill missing numerical values with mean where applicable
cars_df['torque'] = cars_df['torque'].fillna(cars_df['torque'].mean())
cars_df['performance_0_100_km/h'] = cars_df['performance_0_100_km/h'].fillna(cars_df['performance_0_100_km/h'].mean())

# Drop unnecessary columns
cars_df.drop(columns=['cc/battery_capacity', 'performance0_-_100_km/h'], inplace=True)

# Function to categorize car types
def categorize_car_type(car_name):
    car_name = car_name.lower()
    if "suv" in car_name or "crossover" in car_name:
        return "SUV"
    elif "sedan" in car_name or "saloon" in car_name:
        return "Sedan"
    elif "coupe" in car_name or "convertible" in car_name:
        return "Coupe"
    elif "truck" in car_name or "pickup" in car_name:
        return "Truck"
    elif "van" in car_name or "mpv" in car_name:
        return "Van"
    elif "bus" in car_name:
        return "Bus"
    else:
        return "Other"

cars_df['car_types'] = cars_df['cars_names'].apply(categorize_car_type)

# Currency Conversion
conversion_rate = 1.35  # 1 USD = 1.35 CAD
cars_df['cars_prices_cad'] = (cars_df['cars_prices'] * conversion_rate).round(1)  
cars_df['cars_prices_usd'] = cars_df['cars_prices'].round(1)  

# Display first few rows
cars_df.head()

,company_names,cars_names,engines,horsepower,total_speed,cars_prices,fuel_types,seats,torque,cc_battery_capacity,battery_capacity_kwh,cc_capacity,is_electric,performance_0_100_km/h,car_types,cars_prices_cad,cars_prices_usd
0,FERRARI,SF90 STRADALE,V8,963.0,340.0,1100000.0,plug in hyrbrid,2.0,800.0,3990 cc,0.0,3990.0,0,2.5,Other,1485000.0,1100000.0
1,ROLLS ROYCE,PHANTOM,V12,563.0,250.0,460000.0,Petrol,5.0,900.0,6749 cc,0.0,6749.0,0,5.3,Other,621000.0,460000.0
2,Ford,KA+,1.2L Petrol,70.0,165.0,12000.0,Petrol,5.0,100.0,"1,200 cc",0.0,200.0,0,10.5,Other,16200.0,12000.0
3,MERCEDES,GT 63 S,V8,630.0,250.0,161000.0,Petrol,4.0,900.0,"3,982 cc",0.0,982.0,0,3.2,Other,217350.0,161000.0
4,AUDI,AUDI R8 Gt,V10,602.0,320.0,253290.0,Petrol,2.0,560.0,"5,204 cc",0.0,204.0,0,3.6,Other,341941.5,253290.0


In [5]:
def categorize_car_type(row):
    fuel_type = str(row['fuel_types']).lower()
    seats = row['seats']
    horsepower = row['horsepower']
    price = row['cars_prices_usd']
    speed = row['total_speed']
    performance = row['performance_0_100_km/h']
    engines = str(row['engines']).lower()
    company = str(row['company_names']).lower()
    
    # Sports car logic
    if (
        'race' in row['cars_names'].lower() or
        ('v8' in engines and seats <= 4) or
        ('v12' in engines and seats <= 4) or 
        (performance <= 4) or
        (speed >= 240 and (seats <= 4 or pd.isna(seats))) or
        (horsepower >= 450 and (seats <= 4 or pd.isna(seats))) or
        (horsepower >= 350 and (seats <= 2 or pd.isna(seats)))
    ):
        return 'sports'

    # Coupe car logic
    if (
        ('coupe' in row['cars_names'].lower()) or
        (seats <= 2 and horsepower >= 200)
    ):
        return 'coupe'

    # Sedan car logic
    if (
        (2 <= seats <= 5 and horsepower >= 100)
    ):
        return 'sedan'
    
    # Bus/Truck logic
    if (
        ('diesel' in fuel_type and seats > 7) or  # Diesel vehicles with many seats
        'pickup' in row['cars_names'].lower() or  # Diesel vehicles with many seats
        'truck' in row['cars_names'].lower() or    # Explicit truck naming
        'bus' in row['cars_names'].lower()        # Explicit bus naming
    ):
        return 'bus/truck'
    
    # SUV logic
    if (
        (5 <= seats <= 7 and horsepower >= 100)
    ):
        return 'suv'

    # Everything else
    return 'others'

# Apply the new categorization
cars_df['car_types'] = cars_df.apply(categorize_car_type, axis=1)

In [6]:
cars_df["car_types"] = cars_df["car_types"].str.lower()
set(cars_df["car_types"])

{'bus/truck', 'coupe', 'others', 'sedan', 'sports', 'suv'}

In [7]:
cars_df["fuel_types"] = cars_df["fuel_types"].str.lower()
set(cars_df["fuel_types"])

{'cng/petrol',
 'diesel',
 'diesel hybrid',
 'diesel/petrol',
 'electric',
 'hybrid',
 'hybrid (petrol)',
 'hybrid/electric',
 'hybrid/petrol',
 'hydrogen',
 'petrol',
 'petrol (hybrid)',
 'petrol, diesel',
 'petrol, hybrid',
 'petrol/awd',
 'petrol/diesel',
 'petrol/ev',
 'petrol/hybrid',
 'plug in hyrbrid',
 'plug-in hybrid'}

In [8]:
gas_pattern = r'cng|petrol|diesel|awd' 
hybrid_pattern = r'plug-in|hyrbrid|hybrid' 
electric_pattern = r'electric|ev|hydrogen'

# Categorize using numpy.select()
conditions = [
    cars_df['fuel_types'].str.contains(gas_pattern, case=False, regex=True),
    cars_df['fuel_types'].str.contains(hybrid_pattern, case=False, regex=True),
    cars_df['fuel_types'].str.contains(electric_pattern, case=False, regex=True)
]

choices = ['gas', 'hybrid', 'electric']

# Apply the categorization
cars_df['fuel_types_cleaned'] = np.select(conditions, choices, default='unknown')

# Display the cleaned dataframe
set(cars_df['fuel_types_cleaned'])

{'electric', 'gas', 'hybrid'}

In [9]:
# Drop missing values (MCAR handling)
cleaned_cars_df = cars_df.dropna()

In [10]:
# Cleaning battery capacity column
def clean_capacity(s):
    if isinstance(s, str):
        return s.replace(',', '').replace('"', '').strip()
    return s

cleaned_cars_df = cleaned_cars_df.copy()
cleaned_cars_df['cc_battery_capacity'] = cleaned_cars_df['cc_battery_capacity'].apply(clean_capacity)

In [11]:
# Remove redundant columns
cols_to_drop = ['cars_prices', 'fuel_types', 'battery_capacity_kwh', 'cc_capacity', 'is_electric']
cleaned_cars_df = cleaned_cars_df.drop(columns=cols_to_drop)
cleaned_cars_df.head()

,company_names,cars_names,engines,horsepower,total_speed,seats,torque,cc_battery_capacity,performance_0_100_km/h,car_types,cars_prices_cad,cars_prices_usd,fuel_types_cleaned
0,FERRARI,SF90 STRADALE,V8,963.0,340.0,2.0,800.0,3990 cc,2.5,sports,1485000.0,1100000.0,hybrid
1,ROLLS ROYCE,PHANTOM,V12,563.0,250.0,5.0,900.0,6749 cc,5.3,sedan,621000.0,460000.0,gas
2,Ford,KA+,1.2L Petrol,70.0,165.0,5.0,100.0,1200 cc,10.5,others,16200.0,12000.0,gas
3,MERCEDES,GT 63 S,V8,630.0,250.0,4.0,900.0,3982 cc,3.2,sports,217350.0,161000.0,gas
4,AUDI,AUDI R8 Gt,V10,602.0,320.0,2.0,560.0,5204 cc,3.6,sports,341941.5,253290.0,gas


In [12]:
# Check the data type of all columns
str_cols = ['company_names', 'cars_names', 'engines', 'cc_battery_capacity', 'car_types', 'fuel_types_cleaned']
numeric_cols = [col for col in cleaned_cars_df.columns if col not in str_cols]

for col in str_cols:
    non_str_rows = cleaned_cars_df[~cleaned_cars_df[col].apply(lambda x: isinstance(x, str))]
    
    if not non_str_rows.empty:
        print(f"Column '{col}' has non-string values:")
        print(non_str_rows)

for col in numeric_cols:
    non_numeric_rows = cleaned_cars_df[~cleaned_cars_df[col].apply(lambda x: isinstance(x, (int, float)))]

    if not non_numeric_rows.empty:
        print(f"Column '{col}' has non-numeric values:")
        print(non_numeric_rows)

In [13]:
# Fix trailing spaces
cleaned_cars_df['company_names'] = cleaned_cars_df['company_names'].str.strip()

# Standardize inconsistent brands
fix_mappings = {
    'ROLLS ROYCE ': 'ROLLS ROYCE',
    'Nissan': 'NISSAN',
    'Volvo': 'VOLVO',
    'KIA  ': 'KIA',
    'Kia': 'KIA'
}

cleaned_cars_df['company_names'] = cleaned_cars_df['company_names'].replace(fix_mappings)

In [14]:
# Capitalize the 'car_types' and 'fuel_types_cleaned' columns
cleaned_cars_df['car_types'] = cleaned_cars_df['car_types'].str.capitalize()
cleaned_cars_df['fuel_types_cleaned'] = cleaned_cars_df['fuel_types_cleaned'].str.capitalize()

# Save cleaned dataset in "data/processed"
reordered_cols = ['company_names', 'cars_names', 'car_types', 'engines', 'fuel_types_cleaned', 'cc_battery_capacity',
                  'horsepower', 'total_speed', 'performance_0_100_km/h', 'seats', 'torque', 'cars_prices_cad', 'cars_prices_usd']
cleaned_cars_df = cleaned_cars_df[reordered_cols]

cleaned_file_path = os.path.join(processed_dir, "cleaned_cars_df.csv")
cleaned_cars_df.to_csv(cleaned_file_path, index=False)

cleaned_cars_df.head()

,company_names,cars_names,car_types,engines,fuel_types_cleaned,cc_battery_capacity,horsepower,total_speed,performance_0_100_km/h,seats,torque,cars_prices_cad,cars_prices_usd
0,FERRARI,SF90 STRADALE,Sports,V8,Hybrid,3990 cc,963.0,340.0,2.5,2.0,800.0,1485000.0,1100000.0
1,ROLLS ROYCE,PHANTOM,Sedan,V12,Gas,6749 cc,563.0,250.0,5.3,5.0,900.0,621000.0,460000.0
2,Ford,KA+,Others,1.2L Petrol,Gas,1200 cc,70.0,165.0,10.5,5.0,100.0,16200.0,12000.0
3,MERCEDES,GT 63 S,Sports,V8,Gas,3982 cc,630.0,250.0,3.2,4.0,900.0,217350.0,161000.0
4,AUDI,AUDI R8 Gt,Sports,V10,Gas,5204 cc,602.0,320.0,3.6,2.0,560.0,341941.5,253290.0
